In [1]:
import sys
from pathlib import Path
parent_dir = Path.cwd().parent
sys.path.append(str(parent_dir))

# Digital Signature, Transaction

In [2]:
from src.digital_signature.ecc import ECC, secp256k1
from src.blockchain.block import Transaction
import random

In [3]:
private_key  = random.randint(1, secp256k1.n-1)
public_key = secp256k1.scalar_multiply(private_key, secp256k1.G)
private_key, public_key  

(63635013195761682088426421276807138359963226833290839722120054901973816209081,
 (39007289515176154362395163099190455341631783864307478387062495689885957272590,
  77402885057453232246122540030730190135796071044677735630285612154634783567412))

In [4]:
tx = Transaction(sender="Miner Micky", receiver="Miner Moe", amount=100)

In [5]:
tx.hash_transaction()

91469887455764377586877807764597904145762171308828855680586952901254648126450

In [6]:
tx.sign(private_key)
tx.signature

(30969423433249862350475656607143244733969870203776263053964803768589432060797,
 28082543860683927740058550911795349227668437432983567611550149739897776761872)

In [7]:
is_valid = tx.verify(public_key)
is_valid

True

# Block

In [8]:
from src.blockchain.block import Block

In [9]:
tx2 = Transaction(sender="Miner Mike", receiver="Miner Micky", amount=50)
tx3 = Transaction(sender="Miner Mikey", receiver="Miner Micky", amount=60)

our_block = Block(index=0, previous_hash="0", data=[tx, tx2, tx3])

In [10]:
our_block.hash

'88cec605d548c17ef793c6ec5c5cf2f32f056f140d6ce828321de72e08a9f4c9'

In [18]:
our_block.data

In [15]:
def hash_criterion(hash: str):
    return hash.startswith("00")

In [ ]:
# PoW
our_block.mine(hash_criterion)

'0045fa4a9116cad174055fa83e8e3bd6875a7289b2b5112cf737b7f3f858c451'

In [17]:
# Verify the transactions:
public_key_registry = {
    "Miner Micky": secp256k1.scalar_multiply(random.randint(1, secp256k1.n-1), secp256k1.G),
    "Miner Moe": secp256k1.scalar_multiply(random.randint(1, secp256k1.n-1), secp256k1.G),
    "Miner Mike": secp256k1.scalar_multiply(random.randint(1, secp256k1.n-1), secp256k1.G),
    "Miner Mikey": secp256k1.scalar_multiply(random.randint(1, secp256k1.n-1), secp256k1.G)
}

In [ ]:
# Sign the transactions:
tx.sign()

# Blockchain

In [9]:
from src.blockchain.blockchain import Blockchain

In [10]:
our_blockchain = Blockchain(hash_criterion)

In [11]:
our_blockchain.chain[0].data

'Genesis Block'

In [12]:
our_blockchain.add_block_PoW(data="The first block of many", max_iterations=10000)

In [13]:
our_blockchain.chain[1].data

'The first block of many'

# Node

In [14]:
from src.blockchain.node import Node

In [15]:
miner_node = Node(name="Miner Micky", blockchain=our_blockchain)

In [16]:
miner_node.mine_block(data="The memoirs of Miner Micky")

'000e7017dd81e9454c02cee805d5833ee6cf520addc5dbab52932e9ea50b58e6'

# Network

In [17]:
from src.blockchain.network import Network

In [18]:
our_network = Network(blockchain=our_blockchain)

In [19]:
our_network.add_node("Miner Micky")
our_network.add_node("Miner Moe")
our_network.add_node("Miner Jack")

In [20]:
our_network.nodes

{'Miner Micky': <src.blockchain.node.Node at 0x18ee1dac6b0>,
 'Miner Moe': <src.blockchain.node.Node at 0x18ee2109400>,
 'Miner Jack': <src.blockchain.node.Node at 0x18ee210aae0>}

In [21]:
our_network.mine_block_turn_based(data="This is a gem", max_attempts_per_node=100)

Starting turn-based mining...
Miner Micky is attempting to mine...
Miner Moe is attempting to mine...
Miner Jack is attempting to mine...
No node found a valid block. Restarting round...
Miner Micky is attempting to mine...
Miner Moe is attempting to mine...
Miner Jack is attempting to mine...
No node found a valid block. Restarting round...
Miner Micky is attempting to mine...
Miner Moe is attempting to mine...
Miner Jack is attempting to mine...
No node found a valid block. Restarting round...
Miner Micky is attempting to mine...
Miner Moe is attempting to mine...
Miner Jack is attempting to mine...
No node found a valid block. Restarting round...
Miner Micky is attempting to mine...
Miner Moe is attempting to mine...
Miner Jack is attempting to mine...
No node found a valid block. Restarting round...
Miner Micky is attempting to mine...
Miner Moe is attempting to mine...
Miner Jack is attempting to mine...
No node found a valid block. Restarting round...
Miner Micky is attempting to

'000722ad817f62fb608fb0ed524d97dccac0df89a8272deaab052cd8451f16d3'

# Hashing

In [2]:
from src.hashing.sha2 import SHA256

In [3]:
# SHA-256 padding:
sample_message = "kutymutya"
binary_message = SHA256.convert_to_binary_string(sample_message)
binary_message = SHA256.add_padding(binary_message)
blocks = SHA256.split_to_blocks(binary_message)
blocks

['01101011011101010111010001111001011011010111010101110100011110010110000110000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001001000']

In [4]:
SHA256.init_hash_values()

[1779033703,
 3144134277,
 1013904242,
 2773480762,
 1359893119,
 2600822924,
 528734635,
 1541459225]

In [6]:
SHA256.digest("hello") == '2cf24dba5fb0a30e26e83b2ac5b9e29e1b161e5c1fa7425e73043362938b9824'

True